<a href="https://colab.research.google.com/github/jimitxdave/Jimit_INFO5731_Spring2026/blob/main/In_class_exercises_4_Text_Cleaning_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In-Class Assignment — Data Preprocessing & Cleaning (Text)  
**Time:** 20 minutes  |  **Points:** 10  

## Instructions
- This is an individual in-class assignment.  
- Write your code **inside each answer cell**.  
- Print the required outputs.  
- Submit your GitHub/Colab link as instructed by the instructor.


You are given a small dataset of customer support messages as a **TAB-separated text file**:  
- `support_messages.txt`

You will download this file from **Canvas** and upload it to your **Google Colab** notebook.

**How to upload it to your Google Colab notebook?**

1. Download `support_messages.txt` from Canvas.
3. In **the left sidebar**, click the **Files** icon (folder).  
4. Click **Upload** and select `support_messages.txt`.

6. RightAfter uploading, the file will appear in the Colab file list on the left.

6. Right-click the file, copy its path, and paste it into the FILE_PATH variable in Q1.

7. Run Q1 to load the dataset.



> Important: Keep the file name exactly as `support_messages.txt`.


## Questions (Total = 10 points)

### Q1 (1 point) — Load the dataset
Load the TAB-separated file into a pandas DataFrame with columns: `id`, `message`.  
Print: **(a)** `df.shape`, **(b)** `df.head(3)`.

### Q2 (3 points) — Descriptive columns
Add these columns for each message and print the full DataFrame:
- `word_count`: number of words  
- `char_count`: number of characters  
- `num_count`: number of digits (0–9)  
- `upper_word_count`: number of ALL-CAPS words (e.g., `"WHY"`, `"DAMAGED"`)  

### Q3 (3 points) — Clean text
Build a `clean_text(text)` function and create a new column `clean` with these steps **in order**:
1) lowercase  
2) remove punctuation/symbols (keep letters/numbers/spaces)  
3) remove English stopwords (use **nltk** or **sklearn** list)  
4) remove extra spaces  

Print the **original** message and **clean** version for rows `id=1` and `id=4`.

### Q4 (2 points) — Regex extraction
Using RegEx, extract and create two new columns:
- `order_id`: first occurrence of pattern `ORD-####` (case-insensitive; `ord-1060` is valid)  
- `email`: first email address if present (otherwise `None`/`NaN`)  

Print: `id`, `order_id`, `email` for all rows.

### Q5 (1 point) — TF-IDF keywords
Using the `clean` column, compute **TF-IDF** for the messages and print the **top 5 keywords** with the highest **average TF-IDF** across documents.


In [ ]:
# Setup (run this cell first)
import re
import pandas as pd


## Q1 (1 point) — Answer below

In [ ]:
import pandas as pd

# Load TAB-separated file
df = pd.read_csv("support_messages.txt", sep="\t")

# Print shape
print(df.shape)

# Print first 3 rows
print(df.head(3))

(8, 2)
   id                                            message
0   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...
1   2  Refund please!!! I was charged 2 times... invo...
2   3        Great service, thanks! arrived in 2 days :)


## Q2 (3 points) — Answer below

In [ ]:
import re

# word_count
df["word_count"] = df["message"].apply(lambda x: len(str(x).split()))

# char_count
df["char_count"] = df["message"].apply(len)

# num_count (count digits)
df["num_count"] = df["message"].apply(lambda x: len(re.findall(r"\d", x)))

# upper_word_count (ALL CAPS words)
df["upper_word_count"] = df["message"].apply(
    lambda x: len(re.findall(r"\b[A-Z]{2,}\b", x))
)

print(df)


   id                                            message  word_count  \
0   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...          12   
1   2  Refund please!!! I was charged 2 times... invo...          11   
2   3        Great service, thanks! arrived in 2 days :)           8   
3   4  WHY is my package DAMAGED??? tracking says del...           8   
4   5  Need to change address: 7421 Frankford Rd Apt ...          12   
5   6  Support ticket: ORD-1050. Call me at (469) 555...           9   
6   7  I can’t login— password reset link not working...          10   
7   8  Item missing from box. pls send replacement!! ...          10   

   char_count  num_count  upper_word_count  
0          73          4                 2  
1          71          9                 2  
2          43          1                 0  
3          55          0                 2  
4          67         13                 1  
5          58         14                 2  
6          76          0            

## Q3 (3 points) — Answer below

In [ ]:
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def clean_text(text):
    # 1. lowercase
    text = text.lower()

    # 2. remove punctuation (keep letters/numbers/spaces)
    text = re.sub(r"[^a-z0-9\s]", "", text)

    # 3. remove stopwords
    words = text.split()
    words = [w for w in words if w not in ENGLISH_STOP_WORDS]

    # 4. remove extra spaces
    text = " ".join(words)

    return text.strip()

df["clean"] = df["message"].apply(clean_text)

# Print rows id = 1 and id = 4
print(df.loc[df["id"].isin([1,4]), ["message", "clean"]])


                                             message  \
0  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...   
3  WHY is my package DAMAGED??? tracking says del...   

                                               clean  
0  hi order late order ord1042 email saraaligmailcom  
3            package damaged tracking says delivered  


## Q4 (2 points) — Answer below

In [ ]:
# Extract first order ID (case-insensitive)
df["order_id"] = df["message"].str.extract(
    r"(ord-\d+)",
    flags=re.IGNORECASE
)

# Extract first email
df["email"] = df["message"].str.extract(
    r"([\w\.-]+@[\w\.-]+\.\w+)"
)

print(df[["id", "order_id", "email"]])


   id  order_id                  email
0   1  ORD-1042     sara.Ali@gmail.com
1   2  ORD-1042                    NaN
2   3       NaN                    NaN
3   4       NaN                    NaN
4   5       NaN                    NaN
5   6  ORD-1050                    NaN
6   7       NaN  mehri.sattari@unt.edu
7   8  ord-1060                    NaN


## Q5 (1 point) — Answer below

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["clean"])

# Average TF-IDF score across documents
mean_tfidf = np.mean(X.toarray(), axis=0)

# Get feature names
features = vectorizer.get_feature_names_out()

# Top 5 keywords
top_indices = mean_tfidf.argsort()[-5:][::-1]
top_keywords = [(features[i], mean_tfidf[i]) for i in top_indices]

print("Top 5 Keywords:")
for word, score in top_keywords:
    print(word, round(score, 4))

Top 5 Keywords:
order 0.1135
ord1042 0.0829
email 0.0795
package 0.0559
says 0.0559


## Grading Checklist
- Q1: correct load + prints  
- Q2: correct counts  
- Q3: cleaning follows the required order + prints for id=1 and id=4  
- Q4: regex extraction works (case-insensitive `ORD-####` and emails)  
- Q5: prints 5 keywords + their scores (rounding is fine)
